In [1]:
# Etapa 1: Query SQL para tab_silver_fato_financeiro_lancamento

from pyspark.sql.functions import col, when, to_date, lit

# Rebase para datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_FINANCEIRO@onelake.dfs.fabric.microsoft.com/lk_financeiro.Lakehouse/Tables/tab_silver_fato_financeiro_lancamento"

tmov = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tmov"
)
tmov.createOrReplaceTempView("tmov")

flan = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoFinanceira.Lakehouse/Tables/flan"
)
flan.createOrReplaceTempView("flan")


df_resultado = spark.sql("""
SELECT 
	f.*, 
	t.CODTMV  
FROM flan f 
    INNER JOIN tmov t 
    ON f.CODCOLIGADA  = t.CODCOLIGADA AND f.IDMOV = t.IDMOV  
""")


# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 7251e26b-980d-48a5-887e-c8a3c5f5a5f4, 3, Finished, Available, Finished)